# Composition maps

In this notebook, we study the behavior of composition maps (Koopman operators) for simple dynamical systems. 

In [1]:
%load_ext nb_mypy
%nb_mypy On
    
%matplotlib ipympl

Version 1.0.3


In [3]:
from scipy.stats import vonmises
from functools import partial
from ipywidgets import widgets, interactive, IntSlider
from nlsa.abstract_algebra import compose_by
from nlsa.dynamics import orbit
from nptyping import NDArray, Shape, Double
from more_itertools import take
from typing import Callable, Generator
import matplotlib.pyplot as plt
import nlsa.function_algebra as fun
import numpy as np

## Working with type hints

We will annotate our code with [type hints](https://peps.python.org/pep-0484/) representing state spaces of dynamical systems and spaces of observables. Including type hints does not affect the runtime behavior of the program, but makes the code be "closer to the math", improving readability and reducing the chance of making logical errors. In addition, type-annotated programs can be checked for correctness by static type checkers such as `mypy` used in this notebook. Below, `R`, `X`, `F`, `X2`, and `F2` are types representing the set of real numbers $\mathbb R$, the one-dimensional periodic domain $X = \mathbb R/\mathbb Z$, the space of functions $\mathcal F = \{f: X \to \mathbb R\}$, the two-dimensional periodic domain $X^2 = \mathbb R^2 / \mathbb Z^2$, and the space of functions $\mathcal F_2 = \{f: X^2 \to \mathbb R\}$, respectively. Note that $X$ is isomorphic to a circle of circumference 1, and $X^2$ is isomorphic to a 2-torus of area 1.   

In [3]:
X = float
R = float
R2 = NDArray[Shape["2"], Double]
F = Callable[[X], R]
X2 = NDArray[Shape["2"], Double]
F2 = Callable[[X2], R]

## Circle rotation

As our first example, we consider a discrete-time rotation generated by the map $\Phi: X \to X$ such that $\Phi(x) = a + x \mod 1$, where $a$ is a real number. We implement the Koopman operator $U : \mathcal F \to \mathcal F$ with $U f = f \circ \Phi$ induced by $\Phi$ as a function `u: F -> F`.

In [4]:
def phi_rot(a: R, x: X) -> X:
    """Circle rotation"""
    y = (x + a) % 1 
    return y


def u_rot(a: R) -> Callable[[F], F]:
    """Composition map induced by circle rotation"""
    phi = partial(phi_rot, a)
    u = compose_by(fun, phi)
    return u

To visualize the action of the Koopman operator, we apply it to an observable $f \in \mathcal F$ given by the [von Mises probability density function](https://en.wikipedia.org/wiki/Von_Mises_distribution) which is the analog of a Gaussian probability density on a periodic domain. Specifically, we have $f(x) = e^{\kappa\cos(2\pi x)}/(2\pi I_0(\kappa))$, where $\kappa$ is a positive concentration parameter (analogous to the inverse variance of a Gaussian), and $I_0$ is the modified Bessel function of the second kind of order 0. We use the implementation of the von Mises distribution provided by the SciPy library. We use the `orbit` function from the `nlsa.dynamics` module to build a Python [generator](https://wiki.python.org/moin/Generators) that implements iterative application of the Koopman operator on this observable. In this case, Koopman operator simply translates $f$ by $-a$ without changing its shape.  

In [5]:
def f_vm(kappa: R) -> Callable[[X], R]:
    """Von Mises probability density"""
    def f(x: X) -> R:
        y = vonmises.pdf(2 * np.pi * x, kappa, loc=np.pi)
        return y
    
    return f


def f_vm_rot(a: R, kappa: R) -> Generator[F, None, None]:
    """Orbit of the von Mises density under the Koopman operator for the 
    circle rotation
    """
    f = f_vm(kappa)
    u = u_rot(a)
    f_orb = orbit(f, u)
    return f_orb

In [6]:
a = np.sqrt(2) / 10 
kappa = 1
n_max = 20
xs = np.linspace(0, 1, 200)
fs = take(n_max, f_vm_rot(a, kappa)) 
fig1 = plt.figure(1)

def plotfunc1(n):
    f = fs[n]
    fxs = f(xs)
    plt.cla()
    plt.plot(xs, fxs)
    plt.xlabel('$x$')
    plt.ylabel('$U^n f(x)$')
    plt.grid(True)
    plt.title(f'Circle rotation by angle $a={a:.3f}$; iteration $n= {n}$')
    plt.show()
    fig1.canvas.draw()
    
interactive(plotfunc1, n=IntSlider(value=0, min=0, max=n_max - 1)) 

interactive(children=(IntSlider(value=0, description='n', max=19), Output()), _dom_classes=('widget-interact',…

## Doubling map

As our next example, we consider the doubling map $\Phi: X \to X$ given by $\Phi(x) = 2 x\mod 1$. We follow a similar approach as in the rotation example to implement the action of the Koopman operator on the von Mises density. Notice that application of the Koopman operator results in a "duplication" of the function from the previous iteration. Thus, unlike the rotation map, as $n$ increases $U^n f$ becomes an increasingly oscillatory function. This indicates that $U^n f$ becomes increasingly difficult to approximate ("predict") as time increases.    

In [200]:
def phi_db(x: X) -> X:
    """Doubling map on the circle"""
    y = 2 * x % 1 
    return y


u_db = compose_by(fun, phi_db)

def f_vm_db(kappa: R) -> Generator[F, None, None]:
    """Orbit of the von Mises density under the Koopman operator for the 
    doubling map
    """
    f = f_vm(kappa)
    f_orb = orbit(f, u_db)
    return f_orb

In [201]:
kappa = 1
n_iter = 7
xs = np.linspace(0, 1, 1000)
fs = take(n_max, f_vm_db(kappa)) 
fig2 = plt.figure(2)

def plotfunc2(n):
    f = fs[n]
    fxs = f(xs)
    plt.cla()
    plt.plot(xs, fxs)
    plt.xlabel('$x$')
    plt.ylabel('$U^n f(x)$')
    plt.grid(True)
    plt.title(f'Doubling map, iteration $n= {n}$')
    plt.show()
    fig2.canvas.draw()
    
    
interactive(plotfunc2, n=IntSlider(value=0, min=0, max=n_max - 1)) 

interactive(children=(IntSlider(value=0, description='n', max=19), Output()), _dom_classes=('widget-interact',…

## Torus rotation

Moving on to dynamical systems in two dimensions, we consider the torus rotation $\Phi: X^2 \to X^2$ given by $\Phi(x) = x + a \mod 1$ with $a = (a_1, a_2) \in \mathbb R^2$. We visualize the action of the corresponding Koopman operator on a two-dimensional von Mises distribution $f$. If $a_1$ and $a_2$ are rationally independent, the system is measure-preserving and ergodic with respect to Lebesgue measure. This means that if we fix a number $c$ strictly smaller than the maximum value of $f$ and track the "blobs" $S_n = \{ x \in X^2 : U^n f(x) \geq c \}$ as $n$ increases, the sets $S_n$ will well-sample in the invariant measure, in the sense that for any reference set $R \subset X^2$ of positive Lebesgue measure, $S_n$ will intersect $R$ on a set of positive measure for some $n$.

In [4]:
def phi_rot2(a: R2, x: X2) -> X2:
    """Torus rotation
    
    This function broadcasts to multidimensional data arrays x whose last 
    dimension is equal to 2.
    """
    
    y = (x + a) % 1.0  
    return y


def u_rot2(a: R2) -> Callable[[F2], F2]:
    """Composition map induced by torus rotation"""
    phi = partial(phi_rot2, a)
    u = compose_by(fun, phi)
    return u


def f_vm2(kappa: R2) -> Callable[[X2], R]:
    """Von Mises probability density on torus"""
    def f(x: X2) -> R:
        y = vonmises.pdf(2 * np.pi * x[..., 0], kappa[0], loc=np.pi) \
            * vonmises.pdf(2 * np.pi * x[..., 1], kappa[1], loc=np.pi)
        return y
    
    return f


def f_vm_rot2(a: R2, kappa: R2) -> Generator[F2, None, None]:
    """Orbit of the von Mises density under the Koopman operator for the 
    torus rotation
    """
    
    f = f_vm2(kappa)
    u = u_rot2(a)
    f_orb = orbit(f, u)
    return f_orb


In [6]:
a = np.array([np.sqrt(2) / 10, np.sqrt(7) / 10]) 
kappa = np.array([1, 1]) 
n_iter = 20
n_plt = 201
xs, ys = np.meshgrid(np.linspace(0, 1, n_plt), np.linspace(0, 1, n_plt))
xys = np.concatenate((xs[:-1,:-1,np.newaxis], ys[:-1,:-1,np.newaxis]), axis=2)
fs = take(n_iter, f_vm_rot2(a, kappa)) 

fig3 = plt.figure(3)

def plotfunc3(n):
    f = fs[n]
    fxys = f(xys)
    plt.cla()
    plt.pcolormesh(xs, ys, fxys)
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.title(f'Torus rotation by angles $a=({a[0]:.3f}, {a[1]:.3f})$; iteration $n= {n}$')
    plt.show()
    fig3.canvas.draw()
    
    
interactive(plotfunc3, n=IntSlider(value=0, min=0, max=n_iter - 1)) 

interactive(children=(IntSlider(value=0, description='n', max=19), Output()), _dom_classes=('widget-interact',…

## Cat map

In [7]:
def phi_cat(x: X2) -> X2:
    """Cat map on the torus
    
    When operating on a single 2D vector, this function is equivalent to the 
    standard matrix-vector product a @ x. The einsum function broadcasts this 
    operation to multidimensional data arrays x, where the last dimension is 
    equal to 2. 
    """
    
    a = np.array([[2, 1], [1, 1]])
    y = np.einsum('ij,...j->...i', a, x) % 1 
    return y


u_cat = compose_by(fun, phi_cat)

def f_vm_cat(kappa: R2) -> Generator[F2, None, None]:
    """Orbit of von Mises density under the Koopman operator for the cat map"""
    
    f = f_vm2(kappa)
    f_orb = orbit(f, u_cat)
    return f_orb

In [9]:
kappa = np.array([1, 1]) 
n_iter = 20
n_plt = 401
xs, ys = np.meshgrid(np.linspace(0, 1, n_plt), np.linspace(0, 1, n_plt))
xys = np.concatenate((xs[:-1,:-1,np.newaxis], ys[:-1,:-1,np.newaxis]), axis=2)
fs = take(n_iter, f_vm_cat(kappa)) 

fig4 = plt.figure(4)

def plotfunc4(n):
    f = fs[n]
    fxys = f(xys)
    plt.cla()
    plt.pcolormesh(xs, ys, fxys)
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.title(f'Cat map; iteration $n= {n}$')
    plt.show()
    fig4.canvas.draw()
    
    
interactive(plotfunc4, n=IntSlider(value=0, min=0, max=n_iter - 1)) 

interactive(children=(IntSlider(value=0, description='n', max=19), Output()), _dom_classes=('widget-interact',…